<a href="https://colab.research.google.com/github/codeteo/greekSalariesML/blob/main/sn_salary_forecast_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIT License Notice

Copyright 2020 Theodoros Ntakouris

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Dataset Notes

This is a notebook that makes predictions for tech salary in Greece.

More information about the origin and the contents of the dataset, on the Social Nerds video on youtube: https://www.youtube.com/watch?v=e-83bz4RhQ4&t=8s

# Required Imports

In [ ]:
!pip install -q sklearn

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from collections import Counter

# Exploration and Data Prep

In [ ]:
csv_url = 'https://gist.githubusercontent.com/ntakouris/a921107d4c2cc1566186c91971e2a664/raw/212e3bffd59d32a1f71df3585cb48fbeee734616/salary_report.csv'
file_path = tf.keras.utils.get_file("salary_report", csv_url)

In [ ]:
df = pd.read_csv(file_path)
df.columns = ['timestamp', 'years_experience', 'devtype', 'languages', 'company_size', 'remote', 'supervisor', 'personal_projects', 'city_residence', 'city_work', 'sex', 'salary']
df = df.drop(columns=['timestamp'])
print(len(df))
df.head(2)

573


,years_experience,devtype,languages,company_size,remote,supervisor,personal_projects,city_residence,city_work,sex,salary
0,4-5,"DevOps, Backend, Frontend","C#, JavaScript",11-50,Και τα δύο,Όχι,Ναι,Αθήνα,Αθήνα,Άντρας,18200
1,1-3,"Backend, Frontend","C#, JavaScript",11-50,Στον χώρο του εργοδότη,Όχι,Όχι,Δράμα,Καβάλα,Άντρας,13100


In [ ]:
print(pd.unique(df['years_experience']))
df = df[df.years_experience != 'Δε δουλεύω ακόμα']
len(df) # 15 who are not working

['4-5' '1-3' '11+' '6-10' 'Δε δουλεύω ακόμα']


557

In [ ]:
UNK = 'unk'

def categorize_clean_columns(df, colname, threshold):
    df[colname] = df[colname].map(lambda x: x.replace(' ', '').split(','))
    c = Counter([item for sublist in df[colname].values for item in sublist])

    map_to_unk = set([el for el in c.elements() if c[el] <= threshold])

    def map_devtype_unk(arr):
      def x_or_unk(x):
        if x in map_to_unk:
          return UNK
        return x

      return [x_or_unk(x) for x in arr]

    df[colname] = df[colname].map(lambda x: map_devtype_unk(x))

    def is_sole_uknown(arr):
      return arr[0] == UNK and len(arr) == 1

    return df[df[colname].map(is_sole_uknown) == False]

In [ ]:
df = categorize_clean_columns(df, 'devtype', 6)
df = categorize_clean_columns(df, 'languages', 6)

len(df) # ~ 20 rows with few entries

538

In [ ]:
def fix_salary(salary):
    salary = salary.replace('.', '')
    salary = salary.replace(',', '')
    salary = salary.replace('€', '')
    salary = int(salary)
    if salary < 4000:
      return salary * 14
    
    return salary

df['salary'] = df['salary'].map(lambda x: fix_salary(x)).astype('float32')

In [ ]:
pd.unique(df['city_residence'].values), pd.unique(df['city_work'].values)

(array(['Αθήνα', 'Δράμα', 'ΠΕΙΡΑΙΑΣ', 'Ηράκλειο', 'Θεσσαλονίκη', 'Ρέθυμνο',
        'Krakow', 'Λεμεσό', 'Amsterdam', 'Χανια', 'Σέρρες', 'London',
        'Κοζανη', 'Πάτρα', 'Nuremberg', 'Glasgow', 'Berlin', 'Vienna',
        'ΣΑΛΑΜΙΝΑ', 'Madrid', 'Καβάλα', 'Κάλυμνος', 'Κομοτηνή', 'Κύπρο',
        'Oxford', 'Βόλος', 'Ξάνθη', 'Ιωάννινα', 'ΚΑΒΑΛΑ', 'Cologne',
        'Λάρισα', 'Chios', 'Karlskrona', 'Κοζάνη', 'Malmö - Sweden',
        'Munich', 'Stockholm', 'Τρίπολη'], dtype=object),
 array(['Αθήνα', 'Καβάλα', 'ΠΕΙΡΑΙΑΣ', 'Ηράκλειο', 'lisbon', 'Θεσσαλονίκη',
        'Stuttgart', 'Αίγιο', 'δεν έχω συγκεκριμένη πολη', nan,
        'Amsterdam', 'London', 'Πάτρα', 'Nuremberg', 'Glasgow', 'Berlin',
        'Δράμα', 'Λευκωσια', 'Madrid', 'Κάλυμνος', 'San Francisco',
        'Κομοτηνή', 'Λάρνακα', 'Oxford', 'Βόλος', 'Zurich', 'Ξάνθη',
        'Ιωάννινα', 'ΚΑΒΑΛΑ', 'Miami', 'Chios', 'Stockholm', 'Κοζάνη',
        'Cambridge', 'Munich'], dtype=object))

In [ ]:
exclude_non_greek_cities = ['lisbon', 'Stuttgart', 'Cambridge','Miami','Zurich','San Francisco', 'Krakow', 'Amsterdam', 'Berlin', 'Vienna', 'Madrid', 'Glasgow',
                            'Kalmar', 'Cologne', 'Nuremberg', 'Munich', 'Stockholm', 'Karlskrona',
                            'Eindhoven', 'London', 'Malmö - Sweden', 'Bratislava', 'δεν έχω συγκεκριμένη πολη', 'Oxford']

df = df[df['city_residence'].map(lambda x: x not in exclude_non_greek_cities)]
df = df[df['city_work'].map(lambda x: x not in exclude_non_greek_cities)]
print(len(df)) # ~ 30 who do not work on greece
df.head(3)

506


,years_experience,devtype,languages,company_size,remote,supervisor,personal_projects,city_residence,city_work,sex,salary
0,4-5,"[DevOps, Backend, Frontend]","[C#, JavaScript]",11-50,Και τα δύο,Όχι,Ναι,Αθήνα,Αθήνα,Άντρας,18200.0
1,1-3,"[Backend, Frontend]","[C#, JavaScript]",11-50,Στον χώρο του εργοδότη,Όχι,Όχι,Δράμα,Καβάλα,Άντρας,13100.0
3,11+,[Backend],"[Java, C#]",11-50,Και τα δύο,Ναι,Όχι,ΠΕΙΡΑΙΑΣ,ΠΕΙΡΑΙΑΣ,Άντρας,43500.0


In [ ]:
df = df.drop(columns=['city_residence', 'city_work'])

In [ ]:
df_na = df[df.isna().any(axis=1)]
df_na

,years_experience,devtype,languages,company_size,remote,supervisor,personal_projects,sex,salary
160,4-5,[Backend],[Java],201-500,Και τα δύο,Όχι,NaN,NaN,21000.0
248,1-3,[Mobileapps],[Java],11-50,Στον χώρο του εργοδότη,Ναι,NaN,NaN,16000.0
266,1-3,[DevOps],"[Python, Ruby, Go, unk]",501+,Στον χώρο του εργοδότη,Όχι,Όχι,NaN,18500.0
431,4-5,[Backend],[Java],101-200,Στον χώρο του εργοδότη,Ναι,Όχι,NaN,22000.0
459,1-3,[Desktopapps],[C#],11-50,Στον χώρο του εργοδότη,Όχι,Ναι,NaN,13000.0
463,4-5,[Mobileapps],[Kotlin],201-500,Στον χώρο του εργοδότη,Όχι,Όχι,NaN,19700.0
484,11+,"[DevOps, Backend, Desktopapps]","[Java, Python, Ruby, Bash]",11-50,Στον χώρο του εργοδότη,Ναι,Όχι,NaN,57000.0
530,6-10,[AI/ML],"[Java, Python, R, unk]",11-50,Στον χώρο του εργοδότη,Ναι,Όχι,NaN,38000.0
554,1-3,[Backend],"[JavaScript, PHP]",51-100,Απομακρυσμένα,Ναι,NaN,NaN,7724.0
566,1-3,"[Backend, Frontend]","[JavaScript, PHP]",11-50,Στον χώρο του εργοδότη,Όχι,NaN,Άντρας,10800.0


In [ ]:
df['personal_projects'] = df['personal_projects'].fillna('Όχι')
df['sex'] = df['sex'].fillna('Άντρας')
df_na = df[df.isna().any(axis=1)]
df_na

,years_experience,devtype,languages,company_size,remote,supervisor,personal_projects,sex,salary


# To tensorflow

In [ ]:
category_columns = ['years_experience', 'company_size', 'remote', 'supervisor', 'personal_projects', 'sex']

for col in category_columns:
    df[f'{col}_xf'] = df[col].astype('category')

df = df.drop(columns=category_columns)

In [ ]:
df.head(3)

,devtype,languages,salary,years_experience_xf,company_size_xf,remote_xf,supervisor_xf,personal_projects_xf,sex_xf
0,"[DevOps, Backend, Frontend]","[C#, JavaScript]",18200.0,4-5,11-50,Και τα δύο,Όχι,Ναι,Άντρας
1,"[Backend, Frontend]","[C#, JavaScript]",13100.0,1-3,11-50,Στον χώρο του εργοδότη,Όχι,Όχι,Άντρας
3,[Backend],"[Java, C#]",43500.0,11+,11-50,Και τα δύο,Ναι,Όχι,Άντρας


In [ ]:
multi_category_columns = ['devtype', 'languages']

def coltitle(col, word):
    return f'{col}_{word}'

for col in multi_category_columns:
    vocab = set([item for sublist in df[col].values for item in sublist])
    for word in vocab:
        df[coltitle(col, word)] = 0
        df[coltitle(col, word)] = df[coltitle(col, word)].astype('int32')
    print(vocab)

{'Backend', 'Desktopapps', 'DevOps', 'AI/ML', 'unk', 'Frontend', 'Mobileapps'}
{'C', 'PHP', 'JavaScript', 'Kotlin', 'Python', 'Ruby', 'C++', 'unk', 'Bash', 'Go', 'Java', 'Swift', 'VisualBasic.NET', 'R', 'C#'}


In [ ]:
for index, row in df.iterrows():
    for col in multi_category_columns:
        words = row[col]
        for word in words:
            df.loc[index, coltitle(col, word)] = 1

bad_tf_scope_names = [('languages_C#', 'languages_Csharp'), ('languages_VisualBasic.NET', 'languages_VisualBasicNET'), ('languages_C++', 'languages_Cpp')]
for before, after in bad_tf_scope_names:
    df[after] = df[before]
    df = df.drop(columns=[before])

df = df.drop(columns=multi_category_columns)
df.head(3)

,salary,years_experience_xf,company_size_xf,remote_xf,supervisor_xf,personal_projects_xf,sex_xf,devtype_Backend,devtype_Desktopapps,devtype_DevOps,devtype_AI/ML,devtype_unk,devtype_Frontend,devtype_Mobileapps,languages_C,languages_PHP,languages_JavaScript,languages_Kotlin,languages_Python,languages_Ruby,languages_unk,languages_Bash,languages_Go,languages_Java,languages_Swift,languages_R,languages_Csharp,languages_VisualBasicNET,languages_Cpp
0,18200.0,4-5,11-50,Και τα δύο,Όχι,Ναι,Άντρας,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,13100.0,1-3,11-50,Στον χώρο του εργοδότη,Όχι,Όχι,Άντρας,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
3,43500.0,11+,11-50,Και τα δύο,Ναι,Όχι,Άντρας,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [ ]:
df.dtypes

salary                       float32
years_experience_xf         category
company_size_xf             category
remote_xf                   category
supervisor_xf               category
personal_projects_xf        category
sex_xf                      category
devtype_Backend                int32
devtype_Desktopapps            int32
devtype_DevOps                 int32
devtype_AI/ML                  int32
devtype_unk                    int32
devtype_Frontend               int32
devtype_Mobileapps             int32
languages_C                    int32
languages_PHP                  int32
languages_JavaScript           int32
languages_Kotlin               int32
languages_Python               int32
languages_Ruby                 int32
languages_unk                  int32
languages_Bash                 int32
languages_Go                   int32
languages_Java                 int32
languages_Swift                int32
languages_R                    int32
languages_Csharp               int32
l

In [ ]:
df.values[0], len(df)

(array([18200.0, '4-5', '11-50', 'Και τα δύο', 'Όχι', 'Ναι', 'Άντρας', 1,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       dtype=object), 506)

In [ ]:
train, val = train_test_split(df, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')

404 train examples
102 validation examples


In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('salary')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

train_ds = df_to_dataset(train, batch_size=32)
eval_ds = df_to_dataset(train, batch_size=32, shuffle=True)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  feature_column_keys = list(feature_batch.keys())
  print('Every feature:', feature_column_keys)

Every feature: ['years_experience_xf', 'company_size_xf', 'remote_xf', 'supervisor_xf', 'personal_projects_xf', 'sex_xf', 'devtype_Backend', 'devtype_Desktopapps', 'devtype_DevOps', 'devtype_AI/ML', 'devtype_unk', 'devtype_Frontend', 'devtype_Mobileapps', 'languages_C', 'languages_PHP', 'languages_JavaScript', 'languages_Kotlin', 'languages_Python', 'languages_Ruby', 'languages_unk', 'languages_Bash', 'languages_Go', 'languages_Java', 'languages_Swift', 'languages_R', 'languages_Csharp', 'languages_VisualBasicNET', 'languages_Cpp']


In [ ]:
feature_columns = []

numeric_features = [x for x in feature_column_keys if ('devtype_' in x or 'languages_' in x)]
categorical_features = [x for x in feature_column_keys if '_xf' in x]

for feature in numeric_features:
    feature_columns.append(feature_column.numeric_column(feature))

for feature in categorical_features:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
      feature, pd.unique(df[feature].values))

    feature_columns.append(feature_column.indicator_column(categorical_column))

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(8, activation='relu'),  
  layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss='mae',
              metrics=['mae'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

model.fit(train_ds,
          validation_data=eval_ds,
          epochs=50,
          callbacks=callbacks)

Epoch 1/50
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
 2/13 [===>..........................] - ETA: 0s - loss: 18187.3770 - mae: 18187.3770WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0053s vs `on_train_batch_end` time: 0.0817s). Check your callbacks.
Consider rewriting this model with the Functional API.
13/13 [==============================] - 1s 43ms/step - loss: 19613.3086 - mae: 19613.3086 - val_loss: 19611.4414 - val_mae: 19611.4414
Epoch 2/50
13/13 [==============================] - 0s 7ms/step - loss: 19603.0918 - mae: 19603.0918 - val_loss: 19576.3770 - val_mae: 19576.3770
Epoch 3/50
13/13 [==============================] - 0s 7ms/step - loss: 19428.7285 - mae: 19428.7285 - val_loss: 18944.5254 - val_mae: 18944.5254
Epoch 4/50
13/13 [==============================] - 0s 7ms/step - loss: 16295.4678 - mae: 16295.4678 - val_loss: 8448.6396 - val_mae: 84

# View Training Logs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./logs

# Predict your own stuff

In [ ]:
# inference

input = {
    'years_experience_xf': '1-3', # [4-5, 1-3, 11+, 6-10]
    'company_size_xf': '11-50', # [11-50, <10, 501+, 51-100, 101-200, 201-500]
    'remote_xf': 'Και τα δύο', # [Και τα δύο, Στον χώρο του εργοδότη, Απομακρυσμένα]
    'supervisor_xf': 'Ναι', # [Όχι, Ναι]
    'personal_projects_xf': 'Ναι' ,  #[Ναι, Όχι]
    'sex_xf': 'Άντρας' , # [Άντρας, Γυναίκα]
    'devtype_Backend': 1 ,              
    'devtype_Desktopapps': 1 ,         
    'devtype_DevOps': 0 ,            
    'devtype_AI/ML': 1 ,              
    'devtype_unk': 0 , # any devtype work that is not included above?       
    'devtype_Frontend': 0 ,        
    'devtype_Mobileapps': 0 ,           
    'languages_C': 0 ,             
    'languages_PHP': 0 ,             
    'languages_JavaScript': 0 ,         
    'languages_Kotlin' : 0 ,            
    'languages_Python': 1 ,            
    'languages_Ruby': 0 ,               
    'languages_Bash': 1 ,             
    'languages_Go': 0 ,             
    'languages_Java': 0 ,            
    'languages_Swift': 0 ,            
    'languages_R': 0 ,            
    'languages_Csharp': 1 ,         
    'languages_VisualBasicNET': 0 ,    
    'languages_Cpp': 0 , 
    'languages_unk': 0 # any language that is not used above ?                
}

input = {k: [v] for k, v in input.items()}

prediction = model(input).numpy()[0][0]
f'Βγάζεις: {prediction} ευρώ το χρόνο'

Consider rewriting this model with the Functional API.


'Βγάζεις: 19137.138671875 ευρώ το χρόνο'